In [66]:
import mne, os
from signal_processing.pre_process import *
from basic.arrange_data import read_files
import numpy as np

mne.set_log_level('error')

**Set the folder of EEG files**

Run the cell to be prompted with an input box for choosing the folder containing the EEG files of interest.

*NB! The path has to be in 'Data\Raw\' folder relative to this notebook.*

In [44]:
exp_folder = r"Eyes Closed/T1" # in case you want to just type the dir here
#exp_folder = input('Experiment folder (e.g., Eyes Open\Baseline):')

raw_folder = r"Data/Raw/LEISURE/"
clean_folder = r"Data/Clean/LEISURE"
dir_inprogress = os.path.join(raw_folder,exp_folder)
export_dir = os.path.join(clean_folder,exp_folder)

file_dirs, subject_names = read_files(dir_inprogress,".bdf")

Files in folder: 1


For getting information about the EEG files

In [45]:
raw_test = mne.io.read_raw_bdf(file_dirs[0],infer_types=True,
                            eog=["EXG1","EXG2","EXG3","EXG4","EXG5","EXG6","EXG7","EXG8"],
                            stim_channel="Status")
print(raw_test.info)

<Info | 7 non-empty values
 bads: []
 ch_names: Fp1, AF3, F7, F3, FC1, FC5, T7, C3, CP1, CP5, P7, P3, Pz, PO3, ...
 chs: 32 EEG, 8 EOG, 1 Stimulus
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 208.0 Hz
 meas_date: 2018-10-31 13:41:07 UTC
 nchan: 41
 projs: []
 sfreq: 1024.0 Hz
>


**Pre-process raw EEG files and save them**

1. Read raw BDF file, set EOG channels, and montage type
2. Filter data with FIR bandpass filter and remove EOG noise
3. Segment data into epochs and conduct artefact rejection and augmentation
4. Attempt to create a folder and save the cleaned signal file in FIF format (only successful if the path is empty!)

Resting state

In [ ]:
for i in range(len(file_dirs)):  
    raw = mne.io.read_raw_bdf(file_dirs[i],infer_types=True,
                              eog=["EXG1","EXG2","EXG3","EXG4","EXG5","EXG6","EXG7","EXG8"],
                              stim_channel="Status")#.drop_channels(["Erg1",
                            #   "B1","B2","B3","B4","B5","B6","B7","B8","B9","B10","B11","B12",
                            #   "B13","B14","B15","B16","B17","B18","B19","B20","B21","B22","B23","B24",
                            #   "B25","B26","B27","B28","B29","B30","B31","B32",
                            #   "C1","C2","C3-1","C4-1","C5","C6","C7","C8","C9","C10","C11","C12",
                            #   "C13","C14","C15","C16","C17","C18","C19","C20","C21","C22","C23","C24",
                            #   "C25","C26","C27","C28","C29","C30","C31","C32",
                            #   "D1","D2","D3","D4","D5","D6","D7","D8","D9","D10","D11","D12",
                            #   "D13","D14","D15","D16","D17","D18","D19","D20","D21","D22","D23","D24",
                            #   "D25","D26","D27","D28","D29","D30","D31","D32"]).rename_channels({'C3-0':'C3','C4-0':'C4'})

    raw = raw.set_montage(mne.channels.make_standard_montage("biosemi32")).load_data()\
             .set_eeg_reference(ref_channels='average')

    # Find stimulus points, i.e. start and end of resting state, and crop the signal
    events = mne.find_events(raw,stim_channel='Status',consecutive=False,output='offset')
    print(events)

    cropped_raw = raw.crop(tmin=events[0][0]/raw.info['sfreq'],tmax=events[-4][0]/raw.info['sfreq'])
    
    # Filter the signal with bandpass filter and remove EOG artefacts with SSP
    filt = filter_raw_data(cropped_raw,l_freq=0.5,h_freq=30,eog_remove=True)

    %matplotlib inline
    epochs = artefact_rejection(filt,subject_names[i],epo_duration=5)

    # For displaying interactive EEG plots
    #%matplotlib qt
    #epochs.plot(n_channels=32,n_epochs=1)

    try:
        os.makedirs(export_dir)
    except FileExistsError:
        pass
    try:
        mne.Epochs.save(epochs,fname='{}/{}_clean-epo.fif'.format(export_dir,subject_names[i]),overwrite=True)
    except FileExistsError:
        pass